## GPT2 Feature Matching Part 2 
<br> 
We load the saved npy file here then build a custom feature-matcher.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, AutoModel
from rich.progress import track

2024-02-29 10:58:12.334021: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-29 10:58:12.493019: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 10:58:12.493047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 10:58:12.521256: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 10:58:12.578527: I tensorflow/core/platform/cpu_feature_guar

### Food Nutrient Data

In [2]:
food_df  = pd.read_csv('/mnt/d/works/paperworks/2023/aging/IEEE-Access-Code/food_nutrient_info_finalized.csv')
food_df.head()

,food_item,serving_size,calories (kcal),protein (g),protein (g).1,protein (g).2,protein (g).3,carbohydrate (g),sugar (g)
0,닭갈비,400,595.61,45.90,25.80,45.90,25.80,44.90,21.20
1,닭꼬치,70,176.72,11.56,8.57,11.56,8.57,13.35,3.15
2,더덕구이,100,184.00,3.10,5.20,3.10,5.20,31.10,11.60
3,소양념갈비구이,300,989.15,60.10,71.60,60.10,71.60,26.20,13.90
4,양념장어구이,150,433.35,30.77,30.56,30.77,30.56,8.80,4.18


### Korean Food Data (Train and Test)

In [3]:
food_train = pd.read_csv('data/food_aging_train.csv')
food_test = pd.read_csv('data/food_aging_test.csv')

In [4]:
food_train.head()

,식품명,식품오타,food_line
0,호떡,하떡,하떡<input>호떡
1,달걀찜(새우젓),새우젓계란찜,새우젓계란찜<input>달걀찜(새우젓)
2,오징어덮밥,오징이덥밥,오징이덥밥<input>오징어덮밥
3,참나물무침,참나물,참나물<input>참나물무침
4,크림소스스파게티,크림스파게티,크림스파게티<input>크림소스스파게티


In [5]:
food_test.head()

,식품명,식품오타,test_input
0,달래나물무침,다래나물무침,다래나물무침<input>다래나물무침
1,깻잎찜,깬닙찜,깬닙찜<input>깬닙찜
2,등심돈가스,등심돈까쓰,등심돈까쓰<input>등심돈까쓰
3,게살죽,게살쭉,게살쭉<input>게살쭉
4,버섯전,버섯즌,버섯즌<input>버섯즌


In [6]:
food_train.isnull().sum()

식품명          0
식품오타         0
food_line    0
dtype: int64

In [7]:
food_test.isnull().sum()

식품명           0
식품오타          0
test_input    0
dtype: int64

In [8]:
print(food_train.shape)
print(food_test.shape)

(2337, 3)
(584, 3)


### Read marked text inputs

In [9]:
train_lines = [str(s) for s in food_train['food_line']]
test_lines = [str(s) for s in food_test['test_input']]
print(train_lines[:10])
print(test_lines[:10])

['하떡<input>호떡', '새우젓계란찜<input>달걀찜(새우젓)', '오징이덥밥<input>오징어덮밥', '참나물<input>참나물무침', '크림스파게티<input>크림소스스파게티', '안심돈까쑤<input>안심돈가스', '쇠고기뭇국<input>쇠고기무국', '채소샌드이치<input>채소샌드위치', '채소보끈빱<input>채소볶음밥', '볶은밥<input>볶음밥']
['다래나물무침<input>다래나물무침', '깬닙찜<input>깬닙찜', '등심돈까쓰<input>등심돈까쓰', '게살쭉<input>게살쭉', '버섯즌<input>버섯즌', '고추튀기<input>고추튀기', '간장닭다리구이<input>간장닭다리구이', '버섯샤부<input>버섯샤부', '닥꼬기냉채<input>닥꼬기냉채', '소고기전골<input>소고기전골']


### Load the pre-collected NPY file

In [10]:
# load batch process completed npy file...
gpt_feats = np.load('gpt_feat_food4.npy')

In [11]:
max_size = gpt_feats.shape[1]
print(max_size)

786432


### Build a Text Search Engine with Empty (Random) Weights

In [12]:
x_gpt = tf.keras.Input(shape=(max_size,))
x = tf.keras.layers.Dense(units=gpt_feats.shape[0], activation='linear', name='fc1', use_bias=False)(x_gpt)   
x_gpt_feature_matcher = tf.keras.models.Model(inputs=x_gpt, outputs=x)

2024-02-29 10:58:33.042740: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-29 10:58:33.131507: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-29 10:58:33.131651: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-29 10:58:33.132890: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-29 10:58:33.133022: I external/local_xla/xla/stream_executor

### Check the NPY file size

In [13]:
gpt_feats.shape

(2337, 786432)

### Get the random weights from the Text Search Engine

In [14]:
temp_weights = x_gpt_feature_matcher.get_weights()

### Check the weight size; it is a transposed size of the NPY file

In [15]:
temp_weights[-1].shape

(786432, 2337)

### Insert the Transposed NPY into the Weights then Set Weights 

In [16]:
temp_weights[-1] = gpt_feats.T 
x_gpt_feature_matcher.set_weights(temp_weights)

### Save Weights and Model

In [17]:
#x_gpt_feature_matcher.save_weights('100-ko-food-featmatching-gpt4.h5')

In [18]:
json_model = x_gpt_feature_matcher.to_json()

#save the model architecture to JSON file
with open('x_gpt_regressor_food4.json', 'w') as json_file:
    json_file.write(json_model)

## Define chosen topics the model can handle

In [28]:
total_text = []
for each in track(train_lines):
    total_text.append(each)

In [19]:
food_test.head()

,식품명,식품오타,test_input
0,달래나물무침,다래나물무침,다래나물무침<input>다래나물무침
1,깻잎찜,깬닙찜,깬닙찜<input>깬닙찜
2,등심돈가스,등심돈까쓰,등심돈까쓰<input>등심돈까쓰
3,게살죽,게살쭉,게살쭉<input>게살쭉
4,버섯전,버섯즌,버섯즌<input>버섯즌


In [20]:
test_samples = {}
real_names = food_test['식품명'].tolist()
for i in range(len(real_names)):
    real_food = real_names[i]
    input_line = test_lines[i]
    test_samples[input_line] = real_food

print(test_samples)


{'다래나물무침<input>다래나물무침': '달래나물무침', '깬닙찜<input>깬닙찜': '깻잎찜', '등심돈까쓰<input>등심돈까쓰': '등심돈가스', '게살쭉<input>게살쭉': '게살죽', '버섯즌<input>버섯즌': '버섯전', '고추튀기<input>고추튀기': '고추튀김', '간장닭다리구이<input>간장닭다리구이': '간장양념닭다리구이', '버섯샤부<input>버섯샤부': '버섯샤브샤브', '닥꼬기냉채<input>닥꼬기냉채': '닭고기냉채', '소고기전골<input>소고기전골': '쇠고기전골', '삼겹살<input>삼겹살': '삼겹살구이', '딱뽀끔탕<input>딱뽀끔탕': '닭볶음탕', '냉이된장찌기<input>냉이된장찌기': '냉이된장찌개', '순두부김치찌기<input>순두부김치찌기': '순두부김치찌개', '가배기<input>가배기': '꽈배기', '쇠고기완자즌<input>쇠고기완자즌': '쇠고기완자전', '우렁된장구<input>우렁된장구': '우렁된장국', '불고기햄버거<input>불고기햄버거': '불고기버거', '소고기주먹밥<input>소고기주먹밥': '쇠고기주먹밥', '해덥밥<input>해덥밥': '회덮밥', '감저사라다<input>감저사라다': '감자샐러드', '소고기채소보끈<input>소고기채소보끈': '쇠고기채소볶음', '대지괴기산즉<input>대지괴기산즉': '돼지고기산적', '바지락조개궁<input>바지락조개궁': '바지락조개국', '오징어티김<input>오징어티김': '오징어튀김', '다시마멋침<input>다시마멋침': '다시마무침', '쇠고기고추장보끈<input>쇠고기고추장보끈': '쇠고기고추장볶음', '닥뽀끔<input>닥뽀끔': '닭볶음', '중국우동<input>중국우동': '우동(중식)', '오이장아지<input>오이장아지': '오이장아찌', '갓낌치<input>갓낌치': '갓김치', '계란찜(우유)<input>계란찜(우유)': '달걀찜(우유)', '우거질해장국<input>우거질해장국': '우거지해장국', '쌀국시<

In [21]:
print(test_lines[:10])

['다래나물무침<input>다래나물무침', '깬닙찜<input>깬닙찜', '등심돈까쓰<input>등심돈까쓰', '게살쭉<input>게살쭉', '버섯즌<input>버섯즌', '고추튀기<input>고추튀기', '간장닭다리구이<input>간장닭다리구이', '버섯샤부<input>버섯샤부', '닥꼬기냉채<input>닥꼬기냉채', '소고기전골<input>소고기전골']


In [22]:
# This model "eaglewatch/gpt2-ko-wikipedia" was trained with "eaglewatch/Korean_Wikipedia_Dataset_for_GPT2_August_2022" dataset from HuggingFace
# this dataset can be downloaded as follow;

#from datasets import load_dataset
#dataset = load_dataset("eaglewatch/Korean_Wikipedia_Dataset_for_GPT2_August_2022")

tokenizer = AutoTokenizer.from_pretrained("eaglewatch/gpt2-ko-wikipedia")
gpt_model = AutoModel.from_pretrained("eaglewatch/gpt2-ko-wikipedia", output_hidden_states=True)

### Text Search Engine Responses for Tests

In [23]:
max_token_length = 1024
input_feats = np.zeros((len(test_lines), max_token_length*768))

for i in track(range(len(test_lines))):
    encoded = tokenizer(test_lines[i], max_length=1024, padding='max_length', truncation=True, return_tensors='pt')
    input_feats[i] = gpt_model(**encoded).last_hidden_state.detach().numpy().reshape(1024*768)

/home/eaglewatch/miniconda3/envs/kobertnlp/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [24]:
import torch

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# start the event
start.record()


prediction = x_gpt_feature_matcher.predict(input_feats)


end.record()

# wait until the event is done
torch.cuda.synchronize()

elapsed_time = start.elapsed_time(end)  # msec
execution_per_sample = (elapsed_time / 1000) / len(test_lines)
print(f"Elapsed time: {execution_per_sample} seconds")  # convert it to sec


19/19 [==============================] - 1s 14ms/step
Elapsed time: 0.005116126491598887 seconds


In [25]:
prediction.shape

(584, 2337)

### Top-K Accuracy

In [26]:
def response_acc(k,total_text):
    total_results = []
    for item in prediction:
        item_results = []
        for index in item.argsort()[-k::][::-1]:
            item_results.append(total_text[index].split('<input>')[1])
            item_set = set(item_results)
        total_results.append(item_set)
    
    count = 0
    for i in range(len(test_lines)):
        target = test_samples[test_lines[i]]
        for item in total_results:
            if target in item:
                    count +=1
                    break
    return (count/len(test_lines))*100

In [29]:
print('TOP 1 accuracy of response: ',response_acc(1,total_text))
print('TOP 3 accuracy of response: ',response_acc(3,total_text))
print('TOP 5 accuracy of response: ',response_acc(5,total_text))

TOP 1 accuracy of response:  65.92465753424658
TOP 3 accuracy of response:  84.58904109589042
TOP 5 accuracy of response:  90.41095890410958


### Text Search Engine Response for One Input

In [30]:
input_keyword = '노각머침<input>'

encoded = tokenizer(input_keyword, max_length=1024, padding='max_length', truncation=True, return_tensors='pt')
temp_feats = gpt_model(**encoded).last_hidden_state.detach().numpy().reshape(1,1024*768)

one_prediction = x_gpt_feature_matcher.predict(temp_feats).reshape(-1)
print(one_prediction.shape)

1/1 [==============================] - 0s 10ms/step
(2337,)


In [31]:
k=5
for index in one_prediction.argsort()[-k:][::-1]:
    print (total_text[index])  

노각머침<input>노각무침
노각무침<input>노각무침
노각물침<input>노각무침
햄샌도이치<input>햄샌드위치
갈치찌게<input>갈치찌개


### Final Response

In [32]:
total_text[one_prediction.argsort()[-5:][::-1][0]].split('<input>')[1]

'노각무침'

### Mapping with Nutrient Info 

In [33]:
def find_food_info(df, target_column, chat_response):
    food_info = df[df[target_column] == chat_response]
    return food_info

In [34]:
chat_response = total_text[one_prediction.argsort()[-5:][::-1][0]].split('<input>')[1]
find_food_info(food_df,'food_item',chat_response)

,food_item,serving_size,calories (kcal),protein (g),protein (g).1,protein (g).2,protein (g).3,carbohydrate (g),sugar (g)
76,노각무침,150,84.0,2.6,1.8,2.6,1.8,14.3,7.1


In [35]:
def calculate_precision_recall_f1(k, total_text, label_dict):
    predictions = []
    for item in prediction:
        item_results = []
        for index in item.argsort()[-k::][::-1]:
            item_results.append(total_text[index].split('<input>')[1])
            item_set = set(item_results)
        predictions.append(item_set)
        
    # create truth label table 
    correct_labels_set = set(label_dict.values())

    # count correct prediction from the prediction
    correct_predictions_count = sum(1 for pred in predictions if any(label in pred for label in correct_labels_set))

    predictions_count = len(predictions)

    total_labels_count = len(correct_labels_set)

    # Precision
    precision = correct_predictions_count / predictions_count if predictions_count else 0

    # Recall and make sure it never go over 1.0
    recall = correct_predictions_count / total_labels_count if total_labels_count else 0
    recall = min(recall, 1.0)

    # F1-Score 
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

    return precision, recall, f1_score

In [36]:
calculate_precision_recall_f1(1, total_text, test_samples) 

(0.7277397260273972, 1.0, 0.8424182358771061)